<!-- Stand-alone notebook? -->



# The time-dependent diffusion equation
<div id="tut:timedep"></div>

Previous examples on using FEniCS to solve the Poisson equation
illustrate that solving linear, stationary PDE problems with the aid
of FEniCS is easy and requires little programming.  FEniCS clearly
automates the spatial discretization by the finite element method. One
can use a separate, one-dimensional finite element method in the
domain as well, but very often, it is easier to just use a finite
difference method, or to formulate the problem as an ODE system and
leave the time-stepping to an ODE solver.

<!-- The solution of -->
<!-- nonlinear problems, as we showed in Section -->
<!-- ref{tut:poisson:nonlinear}, can also be automated (cf. Section -->
<!-- ref{tut:nonlinear:Newton:auto}), but many scientists will prefer to -->
<!-- code the solution strategy of the nonlinear problem themselves and -->
<!-- experiment with various combinations of strategies in difficult -->
<!-- problems. Time-dependent problems are somewhat similar in this -->
<!-- respect: we have to add a time discretization scheme, which is often -->
<!-- quite simple, making it natural to explicitly code the details of the -->
<!-- scheme so that the programmer has full control. -->
<!-- We shall explain how -->
<!-- easily this is accomplished through examples. -->

[hpl 1: Should exemplify all three approaches? With emphasis on
simple finite differences?]

## Variational formulation
<div id="tut:timedep:diffusion1"></div>

Our model problem for time-dependent PDEs reads

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1"></div>

$$
\begin{equation}
{\partial u\over\partial t} = \nabla^2 u + f\hbox{ in }\Omega,
\label{tut:diffusion:pde1} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:bc"></div>

$$
\begin{equation}  
u = u_0\hbox{ on } \partial \Omega,
\label{tut:diffusion:pde1:bc} \tag{2}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:ic"></div>

$$
\begin{equation}  
u = I \mbox{ at } t=0{\thinspace .}
\label{tut:diffusion:pde1:ic} \tag{3}
\end{equation}
$$

Here, $u$ varies with space and time, e.g., $u=u(x,y,t)$ if the spatial
domain $\Omega$ is two-dimensional. The source function $f$ and the
boundary values $u_0$ may also vary with space and time.
The initial condition $I$ is a function of space only.

A straightforward approach to solving time-dependent PDEs by the
finite element method is to first discretize the time derivative by a
finite difference approximation, which yields a recursive set of
stationary problems, and then turn each stationary problem into a
variational formulation.

Let superscript $k$ denote a quantity at time $t_k$, where $k$ is an
integer counting time levels. For example, $u^k$ means $u$ at time
level $k$.  A finite difference discretization in time first consists
in sampling the PDE at some time level, say $k$:

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:tk"></div>

$$
\begin{equation} {\partial \over\partial t}u^k = \nabla^2 u^k + f^k{\thinspace .}
\label{tut:diffusion:pde1:tk} \tag{4}
\end{equation}
$$

The time-derivative can be approximated by a finite difference.
For simplicity and stability reasons we choose a
simple backward difference:

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:BE"></div>

$$
\begin{equation} {\partial \over\partial t}u^k\approx {u^k - u^{k-1}\over{{\Delta t}}},
\label{tut:diffusion:BE} \tag{5}
\end{equation}
$$

where ${\Delta t}$ is the time discretization parameter.
Inserting [(5)](#tut:diffusion:BE) in [(4)](#tut:diffusion:pde1:tk) yields

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:BE"></div>

$$
\begin{equation}
{u^k - u^{k-1}\over{{\Delta t}}} = \nabla^2 u^k + f^k{\thinspace .}
\label{tut:diffusion:pde1:BE} \tag{6}
\end{equation}
$$

This is our time-discrete version of the diffusion PDE
[(1)](#tut:diffusion:pde1).

We may reorder [(6)](#tut:diffusion:pde1:BE) so
that the left-hand side contains the terms with the unknown $u^k$ and
the right-hand side contains computed terms only. The result
is a recursive set of spatial
(stationary) problems for $u^k$ (assuming $u^{k-1}$ is known from
computations at the previous time level):

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:u0"></div>

$$
\begin{equation}
u^0 = I, \label{tut:diffusion:pde1:u0} \tag{7}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:uk"></div>

$$
\begin{equation}  
u^k - {{\Delta t}}\nabla^2 u^k =  u^{k-1} + {{\Delta t}} f^k,\quad k=1,2,\ldots
\label{tut:diffusion:pde1:uk} \tag{8}
\end{equation}
$$

Given $I$, we can solve for $u^0$, $u^1$, $u^2$, and so on.

As an alternative to [(8)](#tut:diffusion:pde1:uk), which can be
convenient in implementations, we may collect
all terms on one side of the equality sign:

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:uk2"></div>

$$
\begin{equation}
u^k - {{\Delta t}}\nabla^2 u^k -  u^{k-1} - {{\Delta t}} f^k = ,\quad k=1,2,\ldots
\label{tut:diffusion:pde1:uk2} \tag{9}
\end{equation}
$$

We use a finite element method to solve
[(7)](#tut:diffusion:pde1:u0) and either of the equations
[(8)](#tut:diffusion:pde1:uk) or [(9)](#tut:diffusion:pde1:uk2).  This
requires turning the equations into weak forms.  As usual, we multiply
by a test function $v\in \hat V$ and integrate second-derivatives by
parts. Introducing the symbol $u$ for $u^k$ (which is natural in the
program), the resulting weak form can be conveniently written in
the standard notation:

$$
a_0(u,v)=L_0(v),
$$

for
[(7)](#tut:diffusion:pde1:u0). The formulation [(8)](#tut:diffusion:pde1:uk)
gives rise to

$$
a(u,v)=L(v),
$$

where

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:a0"></div>

$$
\begin{equation}
a_0(u,v) = \int_\Omega uv {\, \mathrm{d}x}, \label{tut:diffusion:pde1:a0} \tag{10}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:L0"></div>

$$
\begin{equation}  
L_0(v) = \int_\Omega Iv {\, \mathrm{d}x}, \label{tut:diffusion:pde1:L0} \tag{11}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:a"></div>

$$
\begin{equation}  
a(u,v) = \int_\Omega\left( uv + {{\Delta t}}
\nabla u\cdot \nabla v\right) {\, \mathrm{d}x}, \label{tut:diffusion:pde1:a} \tag{12}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:L"></div>

$$
\begin{equation}  
L(v) = \int_\Omega \left(u^{k-1} + {{\Delta t}}  f^k\right)v {\, \mathrm{d}x}{\thinspace .}
\label{tut:diffusion:pde1:L} \tag{13}
\end{equation}
$$

The alternative form [(9)](#tut:diffusion:pde1:uk2) has a formulation

$$
F(u,v) = 0,
$$

where

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:F"></div>

$$
\begin{equation}
F = \int_\Omega\left( uv + {{\Delta t}}
\nabla u\cdot \nabla v -
\left(u^{k-1} - {{\Delta t}}  f^k\right)v\right) {\, \mathrm{d}x}{\thinspace .}
\label{tut:diffusion:pde1:F} \tag{14}
\end{equation}
$$

The continuous variational problem is to find
$u^0\in V$ such that $a_0(u^0,v)=L_0(v)$ holds for all $v\in\hat V$,
and then find $u^k\in V$
such that $a(u^k,v)=L(v)$ for all $v\in\hat V$,
or alternatively, $F(u^k,v)=0$ for all $v\in\hat V$,
$k=1,2,\ldots$.

Approximate solutions in space are found by restricting the functional
spaces $V$ and $\hat V$ to finite-dimensional spaces, exactly as we
have done in the Poisson problems.  We shall use the symbol $u$ for
the finite element approximation at time $t_k$. In case we need to
distinguish this space-time discrete approximation from the exact
solution of the continuous diffusion problem, we use ${u_{\small\mbox{e}}}$ for the
latter.  By $u^{k-1}$ we mean the finite element approximation of the
solution at time $t_{k-1}$.

Instead of solving [(7)](#tut:diffusion:pde1:u0) by a finite element
method, i.e., projecting $I$ onto $V$ via the problem
$a_0(u,v)=L_0(v)$, we could simply interpolate $u^0$ from $I$. That
is, if $u^0=\sum_{j=1}^N U^0_j\phi_j$, we simply set $U_j=I(x_j,y_j)$,
where $(x_j,y_j)$ are the coordinates of node number $j$. We refer to
these two strategies as computing the initial condition by either
projecting $I$ or interpolating $I$.  Both operations are easy to
compute through one statement, using either the `project` or
`interpolate` function.

## A simple implementation
<div id="tut:timedep:diffusion1:impl"></div>

Our program needs to implement the time stepping explicitly, but can
rely on FEniCS to easily compute $a_0$, $L_0$, $F$, $a$, and $L$, and solve
the linear systems for the unknowns.

### Test problem

Before starting the coding, we shall construct a problem where it is
easy to determine if the calculations are correct. The simple backward
time difference is exact for linear functions, so we decide to have
a linear variation in time. Combining a second-degree polynomial in space
with a linear term in time,

<!-- Equation labels as ordinary links -->
<div id="tut:diffusion:pde1:u0test"></div>

$$
\begin{equation} u = 1 + x^2 + \alpha y^2 + \beta t,
\label{tut:diffusion:pde1:u0test} \tag{15}
\end{equation}
$$

yields a function whose computed values at the nodes will be exact,
regardless of the size of the elements and ${\Delta t}$, as long as the mesh
is uniformly partitioned.  By inserting
[(15)](#tut:diffusion:pde1:u0test) in the PDE problem
[(1)](#tut:diffusion:pde1), it follows that $u_0$ must be given as
[(15)](#tut:diffusion:pde1:u0test) and that $f(x,y,t)=\beta - 2 -
2\alpha$ and $I(x,y)=1+x^2+\alpha y^2$.


### The code

We first create a mesh:

In [1]:
from __future__ import print_function
from fenics import *
import numpy as np

# Create mesh and define function space
nx = ny = 4
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

A new programming issue is how to deal with functions that vary in
space *and time*, such as the boundary condition $u_0$ given by
[(15)](#tut:diffusion:pde1:u0test).  A natural solution is to apply an
`Expression` object with time $t$ as a parameter, in addition to the
parameters $\alpha$ and $\beta$
for `Expression` objects with parameters:

In [2]:
# Create mesh and define function space
alpha = 3; beta = 1.2
u0 = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                alpha=alpha, beta=beta, t=0)

This function expression has the components of `x` as independent
variables, while `alpha`, `beta`, and `t` are parameters.  The
parameters can later be updated as in

```Python
        u0.t = t
```

The essential boundary conditions, along the whole boundary in this case,
are set in the usual way,

In [3]:
def boundary(x, on_boundary):  # define the Dirichlet boundary
    return on_boundary

bc = DirichletBC(V, u0, boundary)

We shall use `u` for the unknown $u$ at the new time level and `u_1`
for $u$ at the previous time level.  The initial value of `u_1`,
implied by the initial condition on $u$, can be computed by either
projecting or interpolating $I$.  The $I(x,y)$ function is available
in the program through `u0`, as long as `u0.t` is zero.  We can then
do

```Python
        u_1 = interpolate(u0, V)
        # or
        u_1 = project(u0, V)
```

In [4]:
u_1 = interpolate(u0, V)

**Projecting versus interpolating the initial condition.**

To actually recover the
exact solution [(15)](#tut:diffusion:pde1:u0test) to machine precision,
it is important not to compute the discrete initial condition by
projecting $I$, but by interpolating $I$ so that the nodal values are
exact at $t=0$ (projection results in approximative values at the
nodes).



We may either define $a$ or $L$ according to the formulas above, or
we may just define $F$ and ask FEniCS to figure out which terms that
go into the bilinear form $a$ and which that go into the linear form
$L$. The latter is convenient, especially in more complicated problems,
so we illustrate that construction:

In [5]:
dt = 0.3      # time step

u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)

F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_1 + dt*f)*v*dx
a, L = lhs(F), rhs(F)

Finally, we perform the time stepping in a loop:

In [6]:
u = Function(V)   # the unknown at a new time level
T = 2             # total simulation time
t = dt

while t <= T:
    u0.t = t
    solve(a == L, u, bc)

    # Verify
    u_e = interpolate(u0, V)
    error = np.abs(u_e.vector().array() -
                   u.vector().array()).max()
    print('error, t=%.2f: %-10.3g' % (t, max_error))
    t += dt
    u_1.assign(u)

**Remember to update expression objects with the current time!**

Inside the time loop,
observe that `u0.t` must be updated before the `solve` statement
to enforce computation of Dirichlet conditions at the
current time level. (The Dirichlet conditions look up the `u0` object
for values.)





## Diffusion of a Gaussian function

### The mathematical problem

Now we want to solve a more physical problem, namely the diffusion of
a Gaussian hill. It means that the initial condition is given by

$$
I(x,y)= e^{-ax^2 - ay^2}
$$

on a domain $[-2,2]\times [2,2]$. A possible value of $a$ is 5.

### Implementation

What are the necessary changes to the previous program?

1. The domain is not the unit square and it needs much higher resolution: `mesh = RectangleMesh(Point(-2,-2), Point(2,2), 30, 30)`.

2. The boundary condition is zero everywhere: `DirichletBC(V, Constant(0), boundary)`.

3. The initial condition is different: `I = Expression('exp(...)')`.

4. The time step should be sufficiently small: `dt = 0.01` or `dt = 0.05`.

5. The right-hand side function `f` is zero: `f = Constant(0)` (just `0`
   will given an error as functions in FEniCS must be `Expression`, `Function`
   (over a mesh) or `Constant`).

6. The end time for the simulation must be longer: `T = 0.8`.

7. The initial condition and the solution inside the time loop should be
   stored to file in VTK format for visualization: `vtkfile << (u, t)`.

8. We can add a `plot(u)` command inside the time loop as well.

The complete program appears below.

In [7]:
from fenics import *
import time

# Create mesh and define function space
nx = ny = 30
mesh = RectangleMesh(Point(-2,-2), Point(2,2), nx, ny)
V = FunctionSpace(mesh, 'P', 1)

# Define boundary conditions
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, Constant(0), boundary)

# Initial condition
I = Expression('exp(-a*pow(x[0],2)-a*pow(x[1],2))', a=5)
u_1 = interpolate(I, V)
u_1.rename('u', 'initial condition')
vtkfile = File('diffusion.pvd')
vtkfile << (u_1, 0.0)
#project(u0, V) will not result in exact solution at the nodes!

dt = 0.01    # time step

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0)

F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_1 + dt*f)*v*dx
a, L = lhs(F), rhs(F)

# Compute solution
u = Function(V)             # the unknown at a new time level
u.rename('u', 'solution')   # name and label for u
T = 0.5                     # total simulation time
t = dt
while t <= T:
    print('time =', t)
    solve(a == L, u, bc)
    vtkfile << (u, float(t))
    plot(u)
    time.sleep(0.3)

    t += dt
    u_1.assign(u)

PLEASE have in-browser visualization!